## Function Timing Decorator

Timing code execution with the Jupyter `%%time` cell magic is fine
when the calculations actually happen when the cell is executed.
But for `dask` operations the calculations are deferred and `%%time` just shows
us the time it takes to add the deferred calculations to the `dask` operations graph -
that's not what we're interested in.

So, we need to do our timing explicitly, for example:

```python
t_start = time.time()
with xarray.open_dataset(ds_path) as ds:
    depth_avgs = ds.vosaline.isel(y=slice(200, 300), x=slice(200,250)).mean(dim="deptht")
t_end = time.time()
print(t_end - t_start)
```
    4.957032203674316


Adding the `t_start`, `t_end`, and `print(...` to every cell gets really annoying.
Instead, we'll create a timing decorator to do that and write the things we want timed
as decorated functions
(ref: https://realpython.com/primer-on-python-decorators/).

The timing decorator is:

In [1]:
def function_timer(func):
    @functools.wraps(func)
    def wrapper_function_timer(*args, **kwargs):
        t_start = time.time()
        return_value = func(*args, **kwargs)
        t_end = time.time()
        print(f"{t_end - t_start}s")
        return return_value

    return wrapper_function_timer

Now we can re-write our explicitly timed call above as a decorated function
and get its execution time printed when we call the function, for example:

```python
@function_timer
def depth_avgs_from_one_file(ds_path):
    with xarray.open_dataset(ds_path) as ds:
        depth_avgs = ds.vosaline.isel(y=slice(200, 300), x=slice(200,250)).mean(dim="deptht")
    return depth_avgs

depth_avgs = depth_avgs_from_one_file(ds_path)
```
    5.186700344085693s
